In [ ]:
pip install snowflake-connector-python pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import snowflake.connector
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Snowflake connection config
conn = snowflake.connector.connect(
account = "PWOMYGF-UI19644",
user = "JATIN1708",
warehouse = "COMPUTE_WH",
database  = "FEATURE_STORE_DB"  ,   # or "CHURN_DB" if you kept it there
schema    = "FEATURE_STORE"  ,      # or "TELCO_SCHEMA"
password="*"
)

# Query final table
query = "SELECT * FROM telco_customer_churn_features"
df = pd.read_sql(query, conn)
conn.close()

# Preview
print("Data shape:", df.shape)
print("Columns:", df.columns.tolist())


<ipython-input-27-704002d53e00>:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data shape: (7032, 38)
Columns: ['GENDER_ENC', 'PARTNER_ENC', 'DEPENDENTS_ENC', 'PHONESERVICE_ENC', 'PAPERLESS_ENC', 'ML_NO', 'ML_YES', 'IS_DSL', 'IS_FIBER', 'OS_NO', 'OS_YES', 'OB_NO', 'OB_YES', 'DP_NO', 'DP_YES', 'TS_NO', 'TS_YES', 'STV_NO', 'STV_YES', 'SM_NO', 'SM_YES', 'CONTRACT_ONE_YEAR', 'CONTRACT_TWO_YEAR', 'PM_MAILED', 'PM_BANK_TRANSFER', 'PM_CREDIT_CARD', 'BUCKET_0_12', 'BUCKET_13_24', 'BUCKET_25_48', 'BUCKET_49_60', 'AVG_MONTHLY_CHARGE', 'CLV_ESTIMATE', 'HIGH_MONTHLY_CHARGE', 'HAS_LONG_TENURE', 'TENURE_ZSCORE', 'MONTHLYCHARGES_ZSCORE', 'TOTALCHARGES_ZSCORE', 'CHURN_ENC']


In [ ]:
# Split features & target
X = df.drop(columns=['CHURN_ENC'])
y = df['CHURN_ENC']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
# Model
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(class_weight='balanced', random_state=42)
clf.fit(X_train, y_train)

# Predict & evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.63      0.50      0.56       374

    accuracy                           0.79      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
}
gs = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc')
gs.fit(X_train, y_train)
print("Best params:", gs.best_params_)
print("Best CV AUC:", gs.best_score_)


Best params: {'max_depth': 5, 'n_estimators': 200}
Best CV AUC: 0.8424573137179516


In [ ]:
# Predict & evaluate
y_pred = gs.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1033
           1       0.50      0.79      0.61       374

    accuracy                           0.74      1407
   macro avg       0.70      0.75      0.71      1407
weighted avg       0.80      0.74      0.75      1407



In [ ]:
feature_names = X_train.columns  # or list(X.columns)
feat_imp = pd.Series(clf.feature_importances_, index=feature_names)
print(feat_imp.sort_values(ascending=False).head(10))


TOTALCHARGES_ZSCORE      0.104284
AVG_MONTHLY_CHARGE       0.104071
CLV_ESTIMATE             0.103193
MONTHLYCHARGES_ZSCORE    0.099103
TENURE_ZSCORE            0.097003
CONTRACT_TWO_YEAR        0.048867
OS_NO                    0.048841
TS_NO                    0.034635
BUCKET_0_12              0.027425
IS_FIBER                 0.027338
dtype: float64


In [ ]:
X_train.columns

Index(['GENDER_ENC', 'PARTNER_ENC', 'DEPENDENTS_ENC', 'PHONESERVICE_ENC',
       'PAPERLESS_ENC', 'ML_NO', 'ML_YES', 'IS_DSL', 'IS_FIBER', 'OS_NO',
       'OS_YES', 'OB_NO', 'OB_YES', 'DP_NO', 'DP_YES', 'TS_NO', 'TS_YES',
       'STV_NO', 'STV_YES', 'SM_NO', 'SM_YES', 'CONTRACT_ONE_YEAR',
       'CONTRACT_TWO_YEAR', 'PM_MAILED', 'PM_BANK_TRANSFER', 'PM_CREDIT_CARD',
       'BUCKET_0_12', 'BUCKET_13_24', 'BUCKET_25_48', 'BUCKET_49_60',
       'AVG_MONTHLY_CHARGE', 'CLV_ESTIMATE', 'HIGH_MONTHLY_CHARGE',
       'HAS_LONG_TENURE', 'TENURE_ZSCORE', 'MONTHLYCHARGES_ZSCORE',
       'TOTALCHARGES_ZSCORE'],
      dtype='object')